# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [23]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [24]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [25]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [26]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [27]:
from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [28]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [29]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

In [30]:
query = "CREATE TABLE IF NOT EXISTS music_app_history "
query = query + "(session_id int, item_in_session int, song text, artist text, length float, \
                  PRIMARY KEY(session_id, item_in_session, song))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query = query + "(user_id int, session_id int, item_in_session int, artist text, first_name text, last_name text, song text, \
                    PRIMARY KEY((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "CREATE TABLE IF NOT EXISTS music_app_history1 "
query = query + "(song text, user_id int, first_name text, item_in_session int, last_name text, session_id int, \
                    PRIMARY KEY(song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

## music_app_history table(for query 1)

####  I used `session_id` and `item_in_session` as partition key and clustering column, repectively. Also added `song` into clustering column to make the partition key unique

## song_playlist_session table(for query 2)

####  I used `user_id` and `item_in_session` as partition key, `item_in_session` as clustering column to sort the result

## music_app_history1 table(for query 3)

#### I used `song` as partition key, `user_id` as clustering column to sort the result

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [33]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query1 = "INSERT INTO music_app_history (session_id, item_in_session, song, artist, length)"
        query1 = query1 + " VALUES (%s, %s, %s, %s, %s)"
        
        query2 = "INSERT INTO song_playlist_session (user_id, session_id, item_in_session, artist, first_name, last_name, song)"
        query2 = query2 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        query3 = "INSERT INTO music_app_history1 (song, user_id, first_name, item_in_session, last_name, session_id)"
        query3 = query3 + " VALUES (%s, %s, %s, %s, %s, %s)"
        
        session.execute(query1, (int(line[8]), int(line[3]), line[9], line[0], float(line[5])))
        session.execute(query2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]))
        session.execute(query3, (line[9], int(line[10]), line[1], int(line[3]), line[4], int(line[8])))

#### Do a SELECT to verify that the data have been inserted into each table

In [34]:
query = "select artist, song, length from music_app_history WHERE session_id = 139"
print("Table: events1\n")
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)
    
query = "select artist, song, first_name, last_name from song_playlist_session WHERE user_id = 9"
print("\nTable: song_playlist_session\n")
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.first_name, row.last_name)
    
query = "select first_name, last_name from music_app_history1 WHERE song = 'Flat 55'"
print("\nTable: music_app_history\n")
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Table: events1

Des'ree You Gotta Be 246.30812072753906
Mr Oizo Flat 55 144.03872680664062
Tamba Trio Quem Quiser Encontrar O Amor 177.18812561035156
The Mars Volta Eriatarka 380.4207763671875
Infected Mushroom Becoming Insane 440.2673034667969
Blue October / Imogen Heap Congratulations 241.3970947265625
Girl Talk Once again 160.15628051757812

Table: song_playlist_session

Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"

Table: music_app_history

Kaylee Summers


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [35]:
query = "select artist, song, length from music_app_history WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)             

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [36]:
query = "select artist, song, first_name, last_name from song_playlist_session WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.first_name, row.last_name) 

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [37]:
query = "select first_name, last_name from music_app_history1 WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [38]:
query = "DROP TABLE music_app_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE music_app_history1"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [39]:
session.shutdown()
cluster.shutdown()